# A step-by-step guide to build an ESPN fantasy football draft tool with python and pandas

In this post you are going to use ESPN's unofficial Fantasy API to get historical draft data with Python. This data includes the draft pick, the player drafted and the fantasy team that drafted said player.
You'll use Pandas to clean up the data and export it out into a CSV file.
From there, you can upload the CSV file with all historical data into Google Sheets and use it to get a sense for the average count of positions drafted for each pick over the years.

Requirements:
- ESPN Fantasy Football League
- Python
- Pandas
- Google Sheets

## Finding the Request URL

Go to your draft recap within your fantasy football league and inspect the page. https://fantasy.espn.com/football/league/draftrecap?seasonId=2021&leagueId=813342

If you tab over to Headers you can see the Request URL that was used to retrieve this data:

In [ ]:
# https://fantasy.espn.com/apis/v3/games/ffl/seasons/2021/segments/0/leagues/813342?view=mDraftDetail&view=mSettings&view=mTeam&view=modular&view=mNav

Note that this is the URL for any request after 2018. If you go back to a fantasy season beyond that you'll notice a slightly different URL.

In [ ]:
# https://fantasy.espn.com/apis/v3/games/ffl/leagueHistory/813342?view=mDraftDetail&view=mSettings&view=mTeam&view=modular&view=mNav&seasonId=2016

Changing the seasonId within the URL string will change the data returned. Swap out the year with the String.Format method. 

To use this method you use curly braces within the text to serve as a placeholder. You then call the string dot format method and pass in the year variable to the method which will replace the placeholder text when you run it. 

In [ ]:
season_id = '2021'
url = "https://fantasy.espn.com/apis/v3/games/ffl/leagueHistory/813342?view=mDraftDetail&view=mSettings&view=mTeam&view=modular&view=mNav&seasonId={}".format(season_id)


The other variable to pay attention to is the leagueId. This is specific to the ESPN league you are in. I will go into more detail in the next section but if your league is private then you need to include additional cookies in the header to signal that you have the proper permissions to get data from this league. Let's do the same thing you did with the season_id for the league_id. Pay attention that the league id comes first within the URL so you need to pass that in first to the string format method.

In [1]:
season_id = '2021'
league_id = '813342'
url = "https://fantasy.espn.com/apis/v3/games/ffl/leagueHistory/{}?view=mDraftDetail&view=mSettings&view=mTeam&view=modular&view=mNav&seasonId={}".format(league_id, season_id)


## Making API Requests with Python

In [2]:
# import necessary modules
import pandas as pd
import requests

You'll use the Python request library to retrieve all the data you need from espn.com. For your purposes you will be using the GET method. To make a GET request, you'll do so with requests.get(). 

There are three parameters you will pass with the GET request, the first is the URL, the second are the cookies and the third is headers. Let's start with the URL

In [ ]:
url = "https://fantasy.espn.com/apis/v3/games/ffl/leagueHistory/{}?view=mDraftDetail&view=mSettings&view=mTeam&view=modular&view=mNav&seasonId={}".format(league_id, season_id)

There are two cookies you need to include for private leagues. Again, these cookies tell ESPN that it is you making the URL call. You're basically authenticating by providing cookies which only YOU can get access to by logging into your ESPN Fantasy league. The two cookies are swid and espn_s2.

In [3]:
espn_cookies = {"swid": "{46299BEC-CE08-4042-9D6F-CB41B89885EE}",
                            "espn_s2": "AEBZoZ5pskd4UFxLIVgLTWjp8HFIwSn18Xhj7K7beiMLDnVmEj35PviMGsDwr8JwxFkVL5Szm%2FDNg5XL4wBkIh1HbVjZfZAe6%2Br7CmM996QrcsNAU0Sir1DBP7THgiEz8My9wXJQPaHpPMYHFeBVeHaO5uQUwDPcKaxc1xg5bHc1MCBXioQ1uGF7JQ5KYHK%2Bz4uRth1Obnfr1gP9KUBM0xgiVk4McqHFW0WtMibtMRhTBKlU%2BNaWHe6LSuCWG8Au6PcQ90C%2BNEpLO7VJd9JeXStA"}

Through my own trial and error I discovered that you need to pass an additional header into this request if you don't want ESPN filtering the response. Your headers should look like this:

In [4]:
headers  = {
    'Connection': 'keep-alive',
    'Accept': 'application/json, text/plain, */*',
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.130 Safari/537.36',
}

If you save the request URL into a url variable you can make the request call and save the response into a variable called r so you can more easily inspect and access the data you want. Let's look at the 2021 season to for my league to start.

In [5]:
season_id = '2021'
league_id = '813342'

In [6]:
# make the request and save the raw json data
url = "https://fantasy.espn.com/apis/v3/games/ffl/leagueHistory/{}?view=mDraftDetail&view=mSettings&view=mTeam&view=modular&view=mNav&seasonId={}".format(league_id, season_id)
r = requests.get(url,
                 headers=headers,
                 cookies=espn_cookies)
espn_raw_data = r.json()

In [7]:
espn_raw_data

[{'draftDetail': {'completeDate': 1631152984851,
   'drafted': True,
   'inProgress': False,
   'picks': [{'autoDraftTypeId': 3,
     'bidAmount': 0,
     'id': 1,
     'keeper': False,
     'lineupSlotId': 2,
     'nominatingTeamId': 0,
     'overallPickNumber': 1,
     'playerId': 3117251,
     'reservedForKeeper': False,
     'roundId': 1,
     'roundPickNumber': 1,
     'teamId': 3,
     'tradeLocked': False},
    {'autoDraftTypeId': 0,
     'bidAmount': 0,
     'id': 2,
     'keeper': False,
     'lineupSlotId': 0,
     'memberId': '{F9B9D1EB-CCC2-4670-8F66-A0E1951C46D8}',
     'nominatingTeamId': 0,
     'overallPickNumber': 2,
     'playerId': 3918298,
     'reservedForKeeper': False,
     'roundId': 1,
     'roundPickNumber': 2,
     'teamId': 10,
     'tradeLocked': False},
    {'autoDraftTypeId': 0,
     'bidAmount': 0,
     'id': 3,
     'keeper': False,
     'lineupSlotId': 0,
     'memberId': '{6E7107EE-2849-4FBE-B107-EE28491FBE26}',
     'nominatingTeamId': 0,
     'overa

Again, you want to access the pick information which is nested within the draft detail. Because you used the historical API call, you have to get the json within a list which means accessing it via the fancy [0].

In [8]:
espn_draft_detail = espn_raw_data[0]
espn_draft_detail

{'draftDetail': {'completeDate': 1631152984851,
  'drafted': True,
  'inProgress': False,
  'picks': [{'autoDraftTypeId': 3,
    'bidAmount': 0,
    'id': 1,
    'keeper': False,
    'lineupSlotId': 2,
    'nominatingTeamId': 0,
    'overallPickNumber': 1,
    'playerId': 3117251,
    'reservedForKeeper': False,
    'roundId': 1,
    'roundPickNumber': 1,
    'teamId': 3,
    'tradeLocked': False},
   {'autoDraftTypeId': 0,
    'bidAmount': 0,
    'id': 2,
    'keeper': False,
    'lineupSlotId': 0,
    'memberId': '{F9B9D1EB-CCC2-4670-8F66-A0E1951C46D8}',
    'nominatingTeamId': 0,
    'overallPickNumber': 2,
    'playerId': 3918298,
    'reservedForKeeper': False,
    'roundId': 1,
    'roundPickNumber': 2,
    'teamId': 10,
    'tradeLocked': False},
   {'autoDraftTypeId': 0,
    'bidAmount': 0,
    'id': 3,
    'keeper': False,
    'lineupSlotId': 0,
    'memberId': '{6E7107EE-2849-4FBE-B107-EE28491FBE26}',
    'nominatingTeamId': 0,
    'overallPickNumber': 3,
    'playerId': 3139

In [9]:
draft_picks = espn_draft_detail['draftDetail']['picks']
draft_picks

[{'autoDraftTypeId': 3,
  'bidAmount': 0,
  'id': 1,
  'keeper': False,
  'lineupSlotId': 2,
  'nominatingTeamId': 0,
  'overallPickNumber': 1,
  'playerId': 3117251,
  'reservedForKeeper': False,
  'roundId': 1,
  'roundPickNumber': 1,
  'teamId': 3,
  'tradeLocked': False},
 {'autoDraftTypeId': 0,
  'bidAmount': 0,
  'id': 2,
  'keeper': False,
  'lineupSlotId': 0,
  'memberId': '{F9B9D1EB-CCC2-4670-8F66-A0E1951C46D8}',
  'nominatingTeamId': 0,
  'overallPickNumber': 2,
  'playerId': 3918298,
  'reservedForKeeper': False,
  'roundId': 1,
  'roundPickNumber': 2,
  'teamId': 10,
  'tradeLocked': False},
 {'autoDraftTypeId': 0,
  'bidAmount': 0,
  'id': 3,
  'keeper': False,
  'lineupSlotId': 0,
  'memberId': '{6E7107EE-2849-4FBE-B107-EE28491FBE26}',
  'nominatingTeamId': 0,
  'overallPickNumber': 3,
  'playerId': 3139477,
  'reservedForKeeper': False,
  'roundId': 1,
  'roundPickNumber': 3,
  'teamId': 11,
  'tradeLocked': False},
 {'autoDraftTypeId': 0,
  'bidAmount': 0,
  'id': 4,
  

From there you can save the data into a draft_picks list and then turn that list into a pandas dataframe with this line of code.

In [10]:
df = pd.DataFrame(draft_picks)
df.sample(5)

,autoDraftTypeId,bidAmount,id,keeper,lineupSlotId,nominatingTeamId,overallPickNumber,playerId,reservedForKeeper,roundId,roundPickNumber,teamId,tradeLocked,memberId
181,0,0,182,False,20,0,182,4241416,False,16,2,1,False,{46299BEC-CE08-4042-9D6F-CB41B89885EE}
31,0,0,32,False,4,0,32,3925357,False,3,8,5,False,{5E0151B0-F4E5-4427-916D-0DEDB5F36AD1}
42,0,0,43,False,23,0,43,4052042,False,4,7,6,False,{CB912C22-3893-4292-912C-223893D2926D}
18,0,0,19,False,0,0,19,4038941,False,2,7,6,False,{CB912C22-3893-4292-912C-223893D2926D}
103,0,0,104,False,20,0,104,4362628,False,9,8,5,False,{5E0151B0-F4E5-4427-916D-0DEDB5F36AD1}


Lastly, all you want are the following three columns:
- overallPickNumber
- playerId
- teamId

So you can create a new data frame and filter on these three columns.

In [11]:
draft_df = df[['overallPickNumber', 'playerId', 'teamId']]
draft_df.sample(5)

,overallPickNumber,playerId,teamId
202,203,2570986,1
53,54,2976499,6
194,195,3122866,11
9,10,3054850,12
63,64,4362887,2


## Getting Player Info

Let's create another request that gets this info and stores it into a pandas dataframe. From there you can eventually join the data frames. The URL for this call can be grabbed from the Headers tab. You can use the string method again to swap out the year

In [12]:
url = "https://fantasy.espn.com/apis/v3/games/ffl/seasons/{}/players?scoringPeriodId=0&view=players_wl".format(season_id)

With the help of an engineering friend smarter than myself, I noticed the following was being passed in the request on the site. Your headers should look like this

In [13]:
custom_headers = {
 'Connection': 'keep-alive',
 'Accept': 'application/json, text/plain, */*',
 'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.130 Safari/537.36',
 'x-fantasy-filter': '{"filterActive":null}',
 'x-fantasy-platform': 'kona-PROD-1dc40132dc2070ef47881dc95b633e62cebc9913',
 'x-fantasy-source': 'kona'
}

From here you can pass in the header argument to your get request, on top of the URL and cookies. Save the response in player_data.

In [14]:
url = "https://fantasy.espn.com/apis/v3/games/ffl/seasons/{}/players?scoringPeriodId=0&view=players_wl".format(season_id)
r = requests.get(url, 
                 cookies=espn_cookies, 
                 headers=custom_headers)
player_data = r.json()

In [15]:
player_data

[{'defaultPositionId': 2,
  'droppable': True,
  'eligibleSlots': [2, 3, 23, 7, 20, 21],
  'firstName': 'Fozzy',
  'fullName': 'Fozzy Whittaker',
  'id': 15755,
  'lastName': 'Whittaker',
  'proTeamId': 0,
  'universeId': 0},
 {'defaultPositionId': 3,
  'droppable': True,
  'eligibleSlots': [3, 4, 5, 23, 7, 20, 21],
  'firstName': 'Jeremy',
  'fullName': 'Jeremy Kelley',
  'id': 15758,
  'lastName': 'Kelley',
  'proTeamId': 3,
  'universeId': 2},
 {'defaultPositionId': 3,
  'droppable': True,
  'eligibleSlots': [3, 4, 5, 23, 7, 20, 21],
  'firstName': 'Antonio',
  'fullName': 'Antonio Gandy-Golden',
  'id': 4029893,
  'lastName': 'Gandy-Golden',
  'ownership': {'percentOwned': 0.02763304188431304},
  'proTeamId': 28,
  'universeId': 2},
 {'defaultPositionId': 2,
  'droppable': True,
  'eligibleSlots': [2, 3, 23, 7, 20, 21],
  'firstName': 'Austin',
  'fullName': 'Austin Johnson',
  'id': 15765,
  'lastName': 'Johnson',
  'proTeamId': 27,
  'universeId': 2},
 {'defaultPositionId': 0,
  

Similar to before, you'll have all the data you need in a mega player_data list. You need to convert it into a dataframe, grab only the columns you want and rename id to player_id to make it easier for to join on it.

In [16]:
df = pd.DataFrame(player_data)
df.sample(5)

,defaultPositionId,droppable,eligibleSlots,firstName,fullName,id,lastName,proTeamId,universeId,ownership
3422,4,True,"[5, 6, 23, 7, 20, 21]",Mason,Mason Schreck,2972331,Schreck,4,1,{'percentOwned': 0.009838325922035608}
941,-1,True,NaN,Dakota,Dakota Dozier,16861,Dozier,16,2,NaN
3719,13,True,"[25, 13, 14, 15, 20, 21]",Brady,Brady Breeze,4038944,Breeze,8,2,{'percentOwned': 0.017933466838030904}
2384,10,True,"[9, 11, 15, 20, 21]",Bud,Bud Dupree,2576702,Dupree,10,1,{'percentOwned': 7.541890946689779}
4946,10,True,"[9, 11, 15, 20, 21]",Marques,Marques Murrell,11049,Murrell,0,0,NaN


In [17]:
# get only needed columns for players
player_df = df[['defaultPositionId','fullName','id','proTeamId']].copy() # can also try loc
# rename id column for inner join
player_df.rename(columns = {'id':'player_id'}, inplace = True)
player_df.sample(5)

,defaultPositionId,fullName,player_id,proTeamId
913,-1,Kevin Pamphile,16835,28
2986,12,Brandon Williams,5484,0
4979,10,Marcus Davenport,3124058,18
4843,1,Cardale Jones,2976299,0
3353,13,Dallin Leavitt,3053801,13


## Getting Team Info

Let's run through the same step you did for the player_info but grab a team_info call instead. I am not going to go into a lot of detail here but drop a comment if you run into any issues.

In [18]:
url = "https://fantasy.espn.com/apis/v3/games/ffl/seasons/{}?view=proTeamSchedules_wl".format(season_id)
r = requests.get(url, 
                 headers=headers,
                 cookies=espn_cookies)
team_data = r.json()
team_data

{'display': True,
 'settings': {'defaultDraftPosition': 170,
  'draftLobbyMinimumLeagueCount': 5,
  'gameNotificationSettings': {'availabilityNotificationsEnabled': True,
   'draftNotificationsEnabled': True,
   'injuryNotificationsEnabled': True,
   'lineupNotificationsEnabled': False,
   'positionEligibilityNotificationsEnabled': False,
   'rosterNewsNotificationsEnabled': True,
   'startBenchNotificationsEnabled': True,
   'tradeNotificationsEnabled': True},
  'gated': False,
  'playerOwnershipSettings': {'firstGameDate': 1631210400000,
   'lastGameDate': 1641751200000,
   'startDate': 1616742000000},
  'proTeams': [{'abbrev': 'Ind',
    'byeWeek': 14,
    'id': 11,
    'location': 'Indianapolis',
    'name': 'Colts',
    'proGamesByScoringPeriod': {'1': [{'awayProTeamId': 26,
       'date': 1631466000000,
       'homeProTeamId': 11,
       'id': 401326318,
       'scoringPeriodId': 1,
       'startTimeTBD': False,
       'statsOfficial': False,
       'validForLocking': True}],
   

In [19]:
team_names = team_data['settings']['proTeams']
team_names

[{'abbrev': 'Ind',
  'byeWeek': 14,
  'id': 11,
  'location': 'Indianapolis',
  'name': 'Colts',
  'proGamesByScoringPeriod': {'1': [{'awayProTeamId': 26,
     'date': 1631466000000,
     'homeProTeamId': 11,
     'id': 401326318,
     'scoringPeriodId': 1,
     'startTimeTBD': False,
     'statsOfficial': False,
     'validForLocking': True}],
   '2': [{'awayProTeamId': 14,
     'date': 1632070800000,
     'homeProTeamId': 11,
     'id': 401326131,
     'scoringPeriodId': 2,
     'startTimeTBD': False,
     'statsOfficial': False,
     'validForLocking': True}],
   '3': [{'awayProTeamId': 11,
     'date': 1632675600000,
     'homeProTeamId': 10,
     'id': 401326372,
     'scoringPeriodId': 3,
     'startTimeTBD': False,
     'statsOfficial': True,
     'validForLocking': True}],
   '4': [{'awayProTeamId': 11,
     'date': 1633280400000,
     'homeProTeamId': 15,
     'id': 401326384,
     'scoringPeriodId': 4,
     'startTimeTBD': False,
     'statsOfficial': True,
     'validForLock

In [20]:
df = pd.DataFrame(team_names)
df.sample(5)

,abbrev,byeWeek,id,location,name,proGamesByScoringPeriod,teamPlayersByPosition,universeId
24,Buf,7,2,Buffalo,Bills,"{'1': [{'awayProTeamId': 23, 'date': 163146600...","{'16': -16002, '14': -14002, '15': -15002}",1
14,SF,6,25,San Francisco,49ers,"{'1': [{'awayProTeamId': 25, 'date': 163146600...","{'16': -16025, '14': -14025, '15': -15025}",2
25,Chi,10,3,Chicago,Bears,"{'1': [{'awayProTeamId': 3, 'date': 1631492400...","{'16': -16003, '14': -14003, '15': -15003}",2
21,Hou,10,34,Houston,Texans,"{'1': [{'awayProTeamId': 30, 'date': 163146600...","{'16': -16034, '14': -14034, '15': -15034}",1
16,TB,9,27,Tampa Bay,Buccaneers,"{'1': [{'awayProTeamId': 6, 'date': 1631233200...","{'16': -16027, '14': -14027, '15': -15027}",2


In [21]:
# get only needed columns for teams
team_df = df[['id', 'location', 'name']].copy()
team_df["team name"] = team_df['location'].astype(str) +" "+ team_df["name"]
# rename in column
team_df.rename(columns = {'id':'team_id'}, inplace = True)
team_df.sample(5)

,team_id,location,name,team name
23,1,Atlanta,Falcons,Atlanta Falcons
2,13,Las Vegas,Raiders,Las Vegas Raiders
6,17,New England,Patriots,New England Patriots
16,27,Tampa Bay,Buccaneers,Tampa Bay Buccaneers
20,33,Baltimore,Ravens,Baltimore Ravens


# Creating Python Functions

In the future you'll want to get the draft details, player info and team info for each historical season. Instead of running the code each time and saving it into its own dataframe, I recommend turning each of these previous sections into a standalone function. For the most part, the function will take in a few inputs, such as the season_id, league_id, headers and cookies, and return a dataframe.

In [22]:
# draft details function
def get_draft_details(league_id, season_id):
    # newest API v3 url
    #url = "https://fantasy.espn.com/apis/v3/games/ffl/seasons/{}/segments/0/leagues/{}?view=mDraftDetail&view=mSettings&view=mTeam&view=modular&view=mNav".format(season_id, league_id)
    url = "https://fantasy.espn.com/apis/v3/games/ffl/leagueHistory/{}?view=mDraftDetail&view=mSettings&view=mTeam&view=modular&view=mNav&seasonId={}".format(league_id, season_id)
    r = requests.get(url,
                     headers=headers,
                     cookies=espn_cookies)
    espn_raw_data = r.json()
    espn_draft_detail = espn_raw_data[0]
    draft_picks = espn_draft_detail['draftDetail']['picks']
    df = pd.DataFrame(draft_picks)
    # get only columns we need in draft detail
    draft_df = df[['overallPickNumber', 'playerId', 'teamId']].copy()
    return draft_df

In [23]:
# get player info
def get_player_info(season_id):
    url = "https://fantasy.espn.com/apis/v3/games/ffl/seasons/{}/players?scoringPeriodId=0&view=players_wl".format(season_id)
    r = requests.get(url,
                    cookies=espn_cookies,
                    headers=custom_headers)
    player_data = r.json()
    df = pd.DataFrame(player_data)
    # get only needed columns for players
    player_df = df[['defaultPositionId','fullName','id','proTeamId']].copy()
    # rename in column
    player_df.rename(columns = {'id':'player_id'}, inplace = True)
    return player_df

In [24]:
# get team information
def get_team_info(season_id):
    url = "https://fantasy.espn.com/apis/v3/games/ffl/seasons/{}?view=proTeamSchedules_wl".format(season_id)
    r = requests.get(url,
                     headers=headers,
                     cookies=espn_cookies)
    team_data = r.json()
    team_names = team_data['settings']['proTeams']
    df = pd.DataFrame(team_names)
    # get only needed columns for teams
    team_df = df[['id', 'location', 'name']].copy()
    team_df["team name"] = team_df['location'].astype(str) +" "+ team_df["name"]
    # rename in column
    team_df.rename(columns = {'id':'team_id'}, inplace = True)
    return team_df

## Merging Dataframes

Right now you have three separate functions all storing information in three separate pandas dataframes. This is a great start but you ultimately want them all in one dataframe so you can export it as a CSV file. To accomplish this, you'll merge dataframes together using certain columns as keys.

In [25]:
import pandas as pd
import requests

In [26]:
# rerun season_id and league_id
season_id = '2021'
league_id = '813342'

In [27]:
# easy access to cookies and headers
espn_cookies = {"swid": "{46299BEC-CE08-4042-9D6F-CB41B89885EE}",
                            "espn_s2": "AEBZoZ5pskd4UFxLIVgLTWjp8HFIwSn18Xhj7K7beiMLDnVmEj35PviMGsDwr8JwxFkVL5Szm%2FDNg5XL4wBkIh1HbVjZfZAe6%2Br7CmM996QrcsNAU0Sir1DBP7THgiEz8My9wXJQPaHpPMYHFeBVeHaO5uQUwDPcKaxc1xg5bHc1MCBXioQ1uGF7JQ5KYHK%2Bz4uRth1Obnfr1gP9KUBM0xgiVk4McqHFW0WtMibtMRhTBKlU%2BNaWHe6LSuCWG8Au6PcQ90C%2BNEpLO7VJd9JeXStA"}

custom_headers = {
 'Connection': 'keep-alive',
 'Accept': 'application/json, text/plain, */*',
 'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.130 Safari/537.36',
 'x-fantasy-filter': '{"filterActive":null}',
 'x-fantasy-platform': 'kona-PROD-1dc40132dc2070ef47881dc95b633e62cebc9913',
 'x-fantasy-source': 'kona'
}

headers = {
 'Connection': 'keep-alive',
 'Accept': 'application/json, text/plain, */*',
 'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.130 Safari/537.36',
}

In [28]:
# get all needed info for the year
draft_df = get_draft_details(league_id, season_id)
player_df = get_player_info(season_id)
team_df = get_team_info(season_id)

After that you can merge the tables with an inner join. To use merge within pandas you need to call out which type of join it will be and which two columns you want to join on. Let's start with the draft_df dataframe and player_df dataframe which we can inner join on our respective player ids:

In [29]:
# merge tables together and save into a new df
df2 = pd.merge(draft_df, player_df, how="inner", left_on="playerId", right_on = "player_id")

In [30]:
df2.sample(5)

,overallPickNumber,playerId,teamId,defaultPositionId,fullName,player_id,proTeamId
94,95,4046692,10,3,Chase Claypool,4046692,23
136,137,14163,5,1,Tyrod Taylor,14163,34
6,7,3916387,8,1,Lamar Jackson,3916387,33
66,67,16733,6,3,Odell Beckham Jr.,16733,14
51,52,3915511,7,1,Joe Burrow,3915511,4


From there you can merge the newly created df2 with the team_df dataframe to add in the team information:

In [31]:
# merge tables together and save into a new df
final_df = pd.merge(df2, team_df, how="inner", left_on="proTeamId", right_on = "team_id")

In [32]:
final_df.sample(5)

,overallPickNumber,playerId,teamId,defaultPositionId,fullName,player_id,proTeamId,team_id,location,name,team name
115,77,8664,4,1,Ryan Fitzpatrick,8664,28,28,Washington,Commanders,Washington Commanders
101,25,3128720,3,2,Nick Chubb,3128720,5,5,Cleveland,Browns,Cleveland Browns
52,121,4036348,3,3,Michael Gallup,4036348,6,6,Dallas,Cowboys,Dallas Cowboys
151,136,3115394,2,3,DJ Chark Jr.,3115394,30,30,Jacksonville,Jaguars,Jacksonville Jaguars
110,87,3120348,12,3,JuJu Smith-Schuster,3120348,23,23,Pittsburgh,Steelers,Pittsburgh Steelers


## Mapping Positions

Within your dataframe you will see a defaultPositionId column. For your purposes of understanding the total count of each position drafted by a certain pick, your life will be easier when you see the actual position instead of the positionId.

To clean this up you can create a position_mapping dictionary which you'll pass into a dataframe replace call. Dictionaries can be used to specify different replacement values for different existing values. The positionIds you want to replace with new string values are

In [33]:
position_mapping = {
 1: 'QB',
 2: 'RB',
 3: 'WR',
 4: 'TE',
 5: 'K',
 16: 'D/ST'
}

You will dot replace the defaultPositionId with this newly created position_mapping dictionary by running the following:

In [34]:
# rename columns and map values for easier consumption. Save to new dataframe
league_draft = final_df.replace({"defaultPositionId": position_mapping})
league_draft.sample(5)

,overallPickNumber,playerId,teamId,defaultPositionId,fullName,player_id,proTeamId,team_id,location,name,team name
0,1,3117251,3,RB,Christian McCaffrey,3117251,29,29,Carolina,Panthers,Carolina Panthers
181,84,4361259,9,QB,Zach Wilson,4361259,20,20,New York,Jets,New York Jets
91,37,15818,9,WR,Keenan Allen,15818,24,24,Los Angeles,Chargers,Los Angeles Chargers
163,190,17359,11,RB,Damien Williams,17359,3,3,Chicago,Bears,Chicago Bears
66,58,16737,12,WR,Mike Evans,16737,27,27,Tampa Bay,Buccaneers,Tampa Bay Buccaneers


## Mapping Fantasy Team Names

Similar to positions, you'll want to do the same for your fantasy team names so you know exactly who drafted who. Merely replace the Team number with the name of your league members.

In [35]:
league_teams = {
 1: 'Team 1',
 2: 'Team 2',
 3: 'Team 3',
 4: 'Team 4',
 5: 'Team 5',
 6: 'Team 6',
 7: 'Team 7',
 8: 'Team 8',
 9: 'Team 9',
 10: 'Team 10',
 11: 'Team 11',
 12: 'Team 12'
}

In [36]:
league_draft_info = league_draft.replace({"teamId": league_teams})
league_draft_info.sample(5)

,overallPickNumber,playerId,teamId,defaultPositionId,fullName,player_id,proTeamId,team_id,location,name,team name
103,63,3059915,Team 12,RB,Kareem Hunt,3059915,5,5,Cleveland,Browns,Cleveland Browns
93,158,12537,Team 1,TE,Jared Cook,12537,24,24,Los Angeles,Chargers,Los Angeles Chargers
195,102,2576434,Team 6,RB,Melvin Gordon III,2576434,7,7,Denver,Broncos,Denver Broncos
60,34,4047650,Team 12,WR,DK Metcalf,4047650,26,26,Seattle,Seahawks,Seattle Seahawks
31,198,-16016,Team 6,D/ST,Vikings D/ST,-16016,16,16,Minnesota,Vikings,Minnesota Vikings


## Renaming Dataframe Columns

Lastly, you can touch up a few of the remaining columns to make them more readable. Before doing that you can filter down to only the columns you need again. 

From there, you can pass in a dictionary directly into the dot replace method with each current column name and desired replacement name.

In [ ]:
league_draft_info.sample(5)

In [37]:
league_draft_final = league_draft_info[['overallPickNumber', 'teamId', 'defaultPositionId', 'fullName', 'team name']]

In [38]:
league_draft_final.rename(columns = {'overallPickNumber':'pick', 'teamId':'geebs_team',
                              'defaultPositionId':'position', 'fullName':'player', 'team name': 'player_team'}, inplace = True)

/Users/jman/projects/geebs/venv/lib/python3.6/site-packages/pandas/core/frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [39]:
league_draft_final.sample(5)

,pick,geebs_team,position,player,player_team
139,39,Team 12,TE,George Kittle,San Francisco 49ers
162,161,Team 5,WR,Darnell Mooney,Chicago Bears
69,110,Team 1,RB,Leonard Fournette,Tampa Bay Buccaneers
63,154,Team 12,K,Jason Myers,Seattle Seahawks
166,120,Team 3,WR,William Fuller V,Miami Dolphins


## Looping Over Every Year

In [40]:
# have all eligible draft years
years = [
 '2012',
 '2013',
 '2014',
 '2015',
 '2016',
 '2017',
 '2018',
 '2019',
 '2020',
 '2021',
]

Next, you need to create an empty dataframe because you are going to loop over every year and append each year's draft details into this dataframe.

In [41]:
# create an empty dataframe to append to
all_drafts_df = pd.DataFrame()

One additional line of code you'll want to add within this for loop is creating a new column which stores the draft year. You don't need this when looking at a single season but when you're looking at 10 seasons of data you'll need a way to groupby each one.

So all in all your for loop should look like this:

In [42]:
# loop over all the years
for year in years:
    print(year)
    # get all needed info for the year
    draft_df = get_draft_details(league_id, year)
    player_df = get_player_info(year)
    team_df = get_team_info(year)
    # merge tables together
    df2 = pd.merge(draft_df, player_df, how="inner", left_on="playerId", right_on = "player_id")
    final_df = pd.merge(df2, team_df, how="inner", left_on="proTeamId", right_on = "team_id")
    # rename columns and map values for easier consumption
    league_draft = final_df.replace({"defaultPositionId": position_mapping})
    league_draft_info = league_draft.replace({"teamId": league_teams})
    league_draft_final = league_draft_info[['overallPickNumber', 'teamId', 'defaultPositionId', 'fullName', 'team name']]
    league_draft_final.rename(columns = {'overallPickNumber':'pick', 'teamId':'geebs_team',
                              'defaultPositionId':'position', 'fullName':'player', 'team name': 'player_team'}, inplace = True)
    league_draft_final['year'] = year
    league_draft_final['year'] = league_draft_final['year'].apply(str)
    all_drafts_df = all_drafts_df.append(league_draft_final)

2012


/Users/jman/projects/geebs/venv/lib/python3.6/site-packages/pandas/core/frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
/Users/jman/projects/geebs/venv/lib/python3.6/site-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/jman/projects/geebs/venv/lib/python3.6/site-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pyd

2013
2014
2015
2016
2017
2018
2019
2020
2021


In [43]:
all_drafts_df.sample(10)

,pick,geebs_team,position,player,player_team,year
12,165,Team 9,QB,Eli Manning,New York Giants,2019
120,87,Team 5,TE,Delanie Walker,Tennessee Titans,2017
148,49,Team 3,QB,Trevor Lawrence,Jacksonville Jaguars,2021
44,67,Team 10,RB,Isaiah Crowell,New York Jets,2017
138,130,Team 12,QB,Jared Goff,Detroit Lions,2021
64,169,Team 11,QB,Case Keenum,Minnesota Vikings,2016
76,12,Team 6,WR,DeAndre Hopkins,Houston Texans,2015
178,77,Team 4,RB,Dion Lewis,Tennessee Titans,2018
87,90,Team 3,WR,Larry Fitzgerald,Arizona Cardinals,2017
75,76,Team 7,WR,Kenny Golladay,New York Giants,2021


## Exporting to CSV

If your intention is to upload this information into Google Sheets then I recommend exporting it as a CSV file. Luckily, pandas provides a really nice and easy way to do this.

In [44]:
# export into CSV
all_drafts_df.to_csv('league_of_geebs_drafts.csv', index=False)

In the future, if you ever want to upload this csv back into a dataframe, you can run the following:

In [ ]:
# read in CSV
# df = pd.read_csv('fantasy_football_drafts.csv')